# How to generate vector field training sets

This notebook explains how to use the components of the `vectorizor` module to
generate 3d vector fields from 3d label volumes.

# Parsing track data

Let's assume track analysis was derived in MatLab and dumped to a JSON format similar
to JSON format to a file that looks like the following:

In [1]:
file_name = "Gata6Nanog1.json"
import json
json_graph = json.load(open(file_name))
json_graph

{'G_based_on_nn_combined': {'Edges': [{'EndNodes': ['000_002', '001_001']},
   {'EndNodes': ['001_001', '002_001']},
   {'EndNodes': ['000_003', '001_002']},
   {'EndNodes': ['001_002', '002_002']},
   {'EndNodes': ['000_005', '001_003']},
   {'EndNodes': ['001_003', '002_005']},
   {'EndNodes': ['000_001', '001_004']},
   {'EndNodes': ['001_004', '002_003']},
   {'EndNodes': ['000_006', '001_005']},
   {'EndNodes': ['001_005', '002_006']},
   {'EndNodes': ['000_004', '001_006']},
   {'EndNodes': ['001_006', '002_004']},
   {'EndNodes': ['000_007', '001_007']},
   {'EndNodes': ['001_007', '002_007']},
   {'EndNodes': ['000_008', '001_008']},
   {'EndNodes': ['001_008', '002_009']},
   {'EndNodes': ['000_009', '001_009']},
   {'EndNodes': ['001_009', '002_008']},
   {'EndNodes': ['002_001', '003_001']},
   {'EndNodes': ['002_002', '003_004']},
   {'EndNodes': ['002_003', '003_005']},
   {'EndNodes': ['002_004', '003_002']},
   {'EndNodes': ['002_005', '003_007']},
   {'EndNodes': ['002_

# Getting track mappings for the timestamps

The vector generation process uses the concept of tracks across timestampe.
These tracks need to be identified using preprocessing as follows.

The following function will identify "tracks" connecting related labels in the JSON data:

In [2]:
from mouse_embryo_labeller.vectorizor import make_tracks_from_haydens_json_graph

timestamp_mapping = make_tracks_from_haydens_json_graph(json_graph)

In [3]:
ts6m = timestamp_mapping[6]
ts6m

{5: 1, 3: 2, 2: 3, 6: 4, 4: 5, 7: 6, 8: 7, 9: 8, 1: 9}

In [4]:
ts7m = timestamp_mapping[7]
ts7m

{5: 1, 1: 2, 3: 3, 6: 4, 9: 5, 7: 6, 2: 7, 8: 8, 4: 9}

The 2 cells above show the track mappings for timestamps 6 and 7.

The mappings indicate, for example, that in timestamp 6 label 9 maps to track 8,
but in timestamp 7 label 8 maps to track 8.  So these two timestamps are talking about
the same nucleus using different label numbers.

These mappings are required for generating vector fields from label volume arrays.

For illustration purposes let's make some fake volume arrays for timestamps 6 and 7 using
inverse mappings:

In [10]:
import numpy as np

inv6 = {track: label for (track, label) in ts6m.items()}
inv7 = {track: label for (track, label) in ts7m.items()}

N = 10
A = np.zeros((N,N,N), dtype=np.int)
B = np.zeros((N,N,N), dtype=np.int)

A[2:5, 1:4, 3:6] = inv6[3]
A[5:7, 7:9, 0:2] = inv6[1]

B[4:7, 3:6, 7:9] = inv7[3]
B[7:9, 4:7, 3:6] = inv7[1]

A[0,0,0] = inv6[4]
B[9,2,9] = inv7[4]

A[1:3,2:5,1:3] = inv6[5]
B[8:10,9,8:10] = inv7[5]

labels6 = A
labels7 = B

# Generating the vectors in one step:

In [11]:
from mouse_embryo_labeller.vectorizor import get_track_vector_field

vector_field = get_track_vector_field(
    old_label_array=labels6, 
    old_labels_to_tracks=ts6m,
    new_label_array=labels7,
    new_labels_to_tracks=ts7m,
    di = (10, 0, 0),  #  xyz offset between A[i,j,k] and A[i+1,j,k]
    dj = (0, 10, 0),  #  xyz offset between A[i,j,k] and A[i,j+1,k]
    dk = (0, 0, 10),  #  xyz offset between A[i,j,k] and A[i,j,k+1]
    )
vector_field.shape

(10, 10, 10, 3)

# What is it doing?

The following breaks down the steps for making the vector field and includes
a small scale 3d illustration of the output.

# First replace labels with track numbers in the arrays

In [12]:
from mouse_embryo_labeller.vectorizor import unify_tracks

(tracks6, tracks7) = unify_tracks(
    A=labels6,  # label array
    A_label_2_track=ts6m,   # mapping of labels in A to track numbers
    B=labels7,  # label array
    B_label_2_track=ts7m,   # mapping of labels in A to track numbers
)

# Then the VectorMaker makes the vectors connecting tracks

In [13]:
from mouse_embryo_labeller.vectorizor import VectorMaker

V = VectorMaker(A, B)
V.scaled_vectors.shape

(10, 10, 10, 3)

# For small test cases you can view the vectors in a widget

In [14]:
W = V.widget()

DualCanvasWidget(status='deferring flush until render')